In [ ]:
!pip install simple_image_download
!pip install cvzone
!pip install mediapipe

In [ ]:
from simple_image_download import simple_image_download as simp
import cvzone
import cv2
from cvzone.SelfiSegmentationModule import SelfiSegmentation
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import requests
from requests.exceptions import Timeout, ConnectionError
import random
from scipy import ndimage
segmentor=SelfiSegmentation()
response = simp.simple_image_download

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = os.path.join(folder,filename) 
        if img is not None:
            images.append(img)
    return images

In [ ]:
breed=['Siberian Huskie','French Bulldog','Dachshund','Dalmatian','park background']

In [ ]:
for b in breed:
  urls=response().urls(b,30)
  for url in urls:
      try:
          res= requests.get(url, timeout=2)
          !wget --no-check-certificate \
          -c --tries=0 --read-timeout=20\
          '{url}' \
          -P '/content/simple_images/{b}'
      except (ConnectionError, Timeout):
          print(url)

In [ ]:
folders=load_images_from_folder('/content/simple_images')
for folder in folders:
  images=load_images_from_folder(folder)
  for image in images:
    if type(cv2.imread(image))!=np.ndarray:
      !rm '{image}'

In [ ]:
class Preprocess():
  def __init__(self, pos, th):
    self.th = th
    self.pos=pos

  def remove_background(self):
    pic=cv2.imread(self.imagePath)
    imgOut=segmentor.removeBG(pic,(0,0,0),threshold=self.th)
    cv2.imwrite('sample' + '.png', imgOut)
    plt.imshow(imgOut)
    img = Image.open('sample.png')
    rgba = img.convert("RGBA")
    datas = rgba.getdata()
      
    newData = []
    for item in datas:
        if item[0] == 0 and item[1] == 0 and item[2] == 0:  
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)  
      
    rgba.putdata(newData)
    rgba.save("sample.png", "PNG")
    return cv2.imread('sample.png',cv2.IMREAD_UNCHANGED)

  def add_background(self):
    background=cv2.imread(self.bgPath)
    background=cv2.resize(background,(56,56))
    pic=self.remove_background()
    pic=cv2.resize(pic,(random.randrange(28)+28,random.randrange(28)+28))
    pic = ndimage.rotate(pic, random.randrange(360))
    res=cvzone.overlayPNG(background,pic,self.pos)
    return res
  
  def draw(self,imagePath, bgPath):
    self.imagePath=imagePath
    self.bgPath=bgPath
    return self.add_background()

In [ ]:
pp=Preprocess([0,0],0.1)
breed=load_images_from_folder('/content/simple_images')
backgrounds=load_images_from_folder('/content/simple_images/park background')
dataset=[]
for b in breed:
  dogs=load_images_from_folder(b)
  test=[]
  while len(test)<1000:
    try:
      pp.pos=[random.randrange(258),random.randrange(258)]
      test.append(pp.draw(random.choice(dogs),random.choice(backgrounds)))
    except:
      if(len(test)%100==0):
        print(len(test))
  dataset.append(test)

In [ ]:
pp=Preprocess([0,0],0.1)
breed=load_images_from_folder('/content/simple_images')
backgrounds=load_images_from_folder('/content/simple_images/park background')
dataset=[]

In [ ]:
b=breed[4]
dogs=load_images_from_folder(b)
test=[]
while len(test)<500:
  try:
    pp.pos=[random.randrange(56),random.randrange(56)]
    test.append(pp.draw(random.choice(dogs),random.choice(backgrounds)))
  except:
    if(len(test)%100==0):
      print(len(test))
dataset.append(test)

In [ ]:
c=0
for data in dataset:
  for image in data:
    c+=1
    cv2.imwrite(f'/content/dataset/{c}.png', image)

In [ ]:
plt.imshow(dataset[3][5])

In [ ]:
dataset=[]
for i in range(4):
  test=[]
  for j in range(1,501):
    test.append(cv2.imread(f'/content/drive/MyDrive/dataset/{i*500+j}.png'))
  dataset.append(test)

In [ ]:
x=[]
y=[]
for i in range(len(dataset)):
    for item in dataset[i]:
        y.append(i)
        x.append(item)
y=np.array(y)
x=np.array(x).astype('float32')
x /= np.amax(x)

In [ ]:
np.save('/content/drive/MyDrive/x', x)
np.save('/content/drive/MyDrive/y', y)